In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [11]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [8]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.categorical_crossentropy(y_true, y_pred)
    return loss1 * 0.7 + loss2 * 0.3

In [9]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

W1007 12:59:44.509602 140650353469184 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 12:59:44.518606 140650353469184 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1007 12:59:44.521697 140650353469184 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1007 12:59:44.977029 140650353469184 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [12]:
model_path = 'mse_cce_models/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min')#, period=5)

In [14]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/350
2360/2360 [==============================] - 20s 9ms/step - loss: 38.5418

Epoch 00001: loss improved from inf to 38.54181, saving model to mse_cce_models/weights01-38.5418.h5
Epoch 2/350
2360/2360 [==============================] - 21s 9ms/step - loss: 38.3649

Epoch 00002: loss improved from 38.54181 to 38.36488, saving model to mse_cce_models/weights02-38.3649.h5
Epoch 3/350
2360/2360 [==============================] - 20s 8ms/step - loss: 38.3501

Epoch 00003: loss improved from 38.36488 to 38.35011, saving model to mse_cce_models/weights03-38.3501.h5
Epoch 4/350
2360/2360 [==============================] - 20s 8ms/step - loss: 38.3457

Epoch 00004: loss improved from 38.35011 to 38.34572, saving model to mse_cce_models/weights04-38.3457.h5
Epoch 5/350
2360/2360 [==============================] - 20s 9ms/step - loss: 30.3287

Epoch 00005: loss improved from 38.34572 to 30.32874, saving model to mse_cce_models/weights05-30.3287.h5
Epoch 6/350
2360/2360 [=================

2360/2360 [==============================] - 19s 8ms/step - loss: 1.5322

Epoch 00047: loss improved from 1.56467 to 1.53223, saving model to mse_cce_models/weights47-1.5322.h5
Epoch 48/350
2360/2360 [==============================] - 19s 8ms/step - loss: 1.7535

Epoch 00048: loss did not improve from 1.53223
Epoch 49/350
2360/2360 [==============================] - 19s 8ms/step - loss: 1.3629

Epoch 00049: loss improved from 1.53223 to 1.36286, saving model to mse_cce_models/weights49-1.3629.h5
Epoch 50/350
2360/2360 [==============================] - 19s 8ms/step - loss: 1.2379

Epoch 00050: loss improved from 1.36286 to 1.23789, saving model to mse_cce_models/weights50-1.2379.h5
Epoch 51/350
2360/2360 [==============================] - 19s 8ms/step - loss: 1.2367

Epoch 00051: loss improved from 1.23789 to 1.23672, saving model to mse_cce_models/weights51-1.2367.h5
Epoch 52/350
2360/2360 [==============================] - 19s 8ms/step - loss: 1.4805

Epoch 00052: loss did not improv

2360/2360 [==============================] - 20s 8ms/step - loss: 0.1603

Epoch 00155: loss did not improve from 0.15414
Epoch 156/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.1781

Epoch 00156: loss did not improve from 0.15414
Epoch 157/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.1452

Epoch 00157: loss improved from 0.15414 to 0.14518, saving model to mse_cce_models/weights157-0.1452.h5
Epoch 158/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.1632

Epoch 00158: loss did not improve from 0.14518
Epoch 159/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.1956

Epoch 00159: loss did not improve from 0.14518
Epoch 160/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.1576

Epoch 00160: loss did not improve from 0.14518
Epoch 161/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.1528

Epoch 00161: loss did not improve from 0.14518
Epoch 162/35

2360/2360 [==============================] - 20s 9ms/step - loss: 0.0968

Epoch 00212: loss improved from 0.10225 to 0.09682, saving model to mse_cce_models/weights212-0.0968.h5
Epoch 213/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.1350

Epoch 00213: loss did not improve from 0.09682
Epoch 214/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.0981

Epoch 00214: loss did not improve from 0.09682
Epoch 215/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.1094

Epoch 00215: loss did not improve from 0.09682
Epoch 216/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.1389

Epoch 00216: loss did not improve from 0.09682
Epoch 217/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.2043

Epoch 00217: loss did not improve from 0.09682
Epoch 218/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.1882

Epoch 00218: loss did not improve from 0.09682
Epoch 219/35

2360/2360 [==============================] - 20s 9ms/step - loss: 0.0691

Epoch 00330: loss did not improve from 0.05436
Epoch 331/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.0689

Epoch 00331: loss did not improve from 0.05436
Epoch 332/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.0667

Epoch 00332: loss did not improve from 0.05436
Epoch 333/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.0649

Epoch 00333: loss did not improve from 0.05436
Epoch 334/350
2360/2360 [==============================] - 20s 9ms/step - loss: 0.0683

Epoch 00334: loss did not improve from 0.05436
Epoch 335/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.0632

Epoch 00335: loss did not improve from 0.05436
Epoch 336/350
2360/2360 [==============================] - 20s 8ms/step - loss: 0.0643

Epoch 00336: loss did not improve from 0.05436
Epoch 337/350
2360/2360 [==============================] - 20s 8ms/st

In [16]:
model_json = lstm_autoencoder.to_json()
filename = input('filename: ') #'last_mse_lstmae' 
with open('mse_cce_models/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('mse_cce_models/weights_' +  filename + '.h5')

filename: latest_lstmae_cce


In [17]:
loaded_model = model_from_json(open('mse_cce_models/' +filename + '.json').read())
loaded_model.load_weights('mse_cce_models/weights_' + filename + '.h5')

In [18]:
loaded_model = lstm_autoencoder

In [19]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

1.1798298669573157


In [ ]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])